<a href="https://colab.research.google.com/github/abuwildanm/Text-Mining/blob/master/Sentiment_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Persiapan

In [580]:
# Import Library Standard
import numpy as np
import pandas as pd

# Import Library Sklearn
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import Library untuk Stemming
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [581]:
# Read Dataset
data = pd.read_excel('https://github.com/abuwildanm/Text-Mining/blob/master/Dataset%20MRT%20Jakarta.xlsx?raw=true')
# data = pd.read_excel('Dataset MRT Jakarta.xlsx')
data.head()

,Komentar,Sumber,Pakar 1 (Firda),Pakar 2 (Adel),Pakar 3 (Faiz),Pakar 4 (Marsha),Positif,Negatif,Hasill akhir
0,"tidak perlu menunggu lama utk menunggu MRT,krn...",Facebook,Positif,Positif,Positif,Positif,4,0,Positif
1,"alhamdulillah baru sempet naik MRt. nyaman, be...",Facebook,Positif,Positif,Positif,Positif,4,0,Positif
2,"Betul, petugas MRT Jakarta sangat ramah terhad...",Twitter,Positif,Positif,Positif,Positif,4,0,Positif
3,Hidup gw jadi lebih nyaman karena gak mikir ma...,Twitter,Positif,Positif,Positif,Positif,4,0,Positif
4,MRT adalah moda raya transportasi yang cepat d...,Twitter,Positif,Positif,Positif,Positif,4,0,Positif


In [582]:
df = data[['Komentar', 'Hasill akhir']].copy()
df.rename(columns={'Komentar':'komentar', 'Hasill akhir':'label'}, inplace=True)
df.head()

,komentar,label
0,"tidak perlu menunggu lama utk menunggu MRT,krn...",Positif
1,"alhamdulillah baru sempet naik MRt. nyaman, be...",Positif
2,"Betul, petugas MRT Jakarta sangat ramah terhad...",Positif
3,Hidup gw jadi lebih nyaman karena gak mikir ma...,Positif
4,MRT adalah moda raya transportasi yang cepat d...,Positif


In [583]:
print('Ukuran Dataset: ', df.shape)
n_document = df.shape[0]

Ukuran Dataset:  (30, 2)


## Preprocessing

### Spell Correction

### Stemming

In [0]:
# Membuat stemmer
stemmerFactory = StemmerFactory()
stemmer = stemmerFactory.create_stemmer()

# Daftar Stopword
stopWordRemoverFactory = StopWordRemoverFactory()
stopwords = stopWordRemoverFactory.get_stop_words()

In [0]:
# Proses Stem
for row in range(n_document):
  df.loc[row, 'komentar'] = stemmer.stem(df.loc[row, 'komentar'])

In [586]:
df.head()

,komentar,label
0,tidak perlu tunggu lama utk tunggu mrt krna ad...,Positif
1,alhamdulillah baru sempet naik mrt nyaman bers...,Positif
2,betul tugas mrt jakarta sangat ramah hadap san...,Positif
3,hidup gw jadi lebih nyaman karena gak mikir ma...,Positif
4,mrt adalah moda raya transportasi yang cepat d...,Positif


## Perhitungan Bobot

In [587]:
vectorizer = CountVectorizer(stop_words=stopwords)
tf = vectorizer.fit_transform(df['komentar'])
tf = pd.DataFrame(tf.toarray(), columns=vectorizer.get_feature_names())
tf

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tau,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


## Pembagian Data

In [0]:
# Pembagian data train dan test
x_train, x_test, y_train, y_test = train_test_split(tf, df['label'], test_size=(10/n_document), stratify=df['label'])
tf_train = pd.DataFrame(x_train, columns=tf.columns)
tf_train['label'] = y_train
tf_test = pd.DataFrame(x_test, columns=tf.columns)
tf_test['label'] = y_test

In [589]:
tf_train

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg,label
29,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,Positif
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Positif
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Positif
20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
16,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Negatif
10,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,Positif
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,Positif


In [590]:
tf_train['label'].value_counts()

Positif    10
Negatif    10
Name: label, dtype: int64

In [591]:
tf_test

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg,label
27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,Positif
24,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
25,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,Negatif
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif


In [592]:
tf_test['label'].value_counts()

Positif    5
Negatif    5
Name: label, dtype: int64

In [593]:
# Ukuran Data Training & Testing
print('Ukuran data train:', tf_train.shape)
print('Ukuran data test:', tf_test.shape)

Ukuran data train: (20, 365)
Ukuran data test: (10, 365)


## Klasifikasi

### Benchmark

In [594]:
mnb = MultinomialNB()
model = mnb.fit(tf_train.iloc[:, :-1], tf_train.iloc[:, -1])
model.predict(tf_test.iloc[:, :-1])

array(['Negatif', 'Positif', 'Positif', 'Positif', 'Negatif', 'Negatif',
       'Negatif', 'Positif', 'Positif', 'Positif'], dtype='<U7')

In [595]:
mnb.score(tf_test.iloc[:, :-1], tf_test.iloc[:, -1])

0.9

### Scratch

In [0]:
class MultinomialNaiveBayes():
  
  def __init__(self, alpha=1.0):
    """Inisialisasi class (constructor)
    
    Keyword Arguments:
        alpha {float} -- smoothing parameter (Laplacian/Lidstone) (default: {1.0})
    """

    self.prior = []
    self.likelihood = []
    self.posterior = []
    self.alpha = alpha

  def calculatePrior(self, label):
    """Untuk menghitung probabilitas kelas (prior)
    
    Arguments:
        label {pandas series} -- label pada data training
    
    Returns:
        pandas series -- nilai probabilitas prior
    """

    prior = label.value_counts() / label.size
    return prior

  def calculateLikelihood(self, data, label):
    """Untuk menghitung probabilitas fitur pada suatu kelas (likelihood)
    
    Arguments:
        data {pandas dataframe} -- data training
        label {pandas series} -- label pada data training
    
    Returns:
        pandas dataframe -- nilai probabilitas likelihood
    """
    
    # Menggabungkan data dengan label
    sparse_data = pd.concat([data, label.reset_index(drop=True)], axis=1)
    # Menghitung jumlah kemunculan suatu kata pada masing-masing kategori
    countWordLabel = sparse_data.groupby('label').sum()
    # Menghitung jumlah total kemunculan semua kata pada masing-masing kategori
    countAllWordLabel = countWordLabel.sum(axis=1)

    likelihood = (countWordLabel + self.alpha).div(countAllWordLabel + data.shape[1], axis=0)
    return likelihood

  # # Option 1
  # def calculatePosterior(self, data):
  #   posterior = []
  #   for i in range(data.shape[0]):
  #     idx_term_test = (data.iloc[i] != 0)
  #     term_test = data.columns[idx_term_test]
  #     feature_test = np.intersect1d(term_test, self.likelihood.columns)
  #     posterior.append(self.prior * self.likelihood[feature_test].prod(axis=1))
    
  #   posterior = pd.DataFrame(posterior)
  #   return posterior

  def calculatePosterior(self, row_data):
    """Untuk menghitung probabilitas suatu kelas yang dipengaruhi oleh fitur-fitur yang ada (posterior)
    
    Arguments:
        row_data {pandas series} -- satu row data
    
    Returns:
        pandas dataframe -- nilai probabilitas posterior
    """

    # Mencari index term pada data testing
    idx_term_test = (row_data != 0)
    # Term yang muncul pada data test
    term_test = row_data.index[idx_term_test]
    # Term yang muncul pada data test dan data train
    feature_test = np.intersect1d(term_test, self.likelihood.columns)

    posterior = self.prior * self.likelihood[feature_test].prod(axis=1)
    return posterior

  def fit(self, x, y):
    """Untuk melakukan training model
    
    Arguments:
        x {pandas dataframe} -- data training
        y {pandas series} -- label pada data training
    """

    # Menghitung prior
    self.prior = self.calculatePrior(y)
    # Menghitung likelihood
    self.likelihood = self.calculateLikelihood(x, y)

  def predict(self, x):
    """Untuk melakukan testing model
    
    Arguments:
        x {pandas dataframe} -- data testing
    
    Returns:
        pandas series -- label hasil prediksi
    """

    # Menghitung posterior
    self.posterior = x.apply(self.calculatePosterior, axis=1)
    # Memprediksi label
    pred = self.posterior.idxmax(axis=1)

    return pred

  def score(self, x, y):
    """Untuk melakukan evaluasi model
    
    Arguments:
        x {pandas dataframe} -- data testing
        y {pandas series} -- label yang sebenarnya pada data testing
    
    Returns:
        float -- hasil evaluasi model
    """
    
    # Memprediksi label
    pred = self.predict(x)
    # Menghitung akurasi
    accuracy = accuracy_score(y, pred)

    return accuracy

In [597]:
mnb = MultinomialNaiveBayes()
mnb.fit(tf_train.iloc[:, :-1], tf_train.iloc[:, -1])
mnb.predict(tf_test.iloc[:, :-1])

27    Positif
4     Negatif
8     Negatif
0     Positif
24    Negatif
25    Positif
17    Negatif
7     Positif
14    Positif
21    Negatif
dtype: object

In [598]:
mnb.score(tf_test.iloc[:, :-1], tf_test.iloc[:, -1])

0.6

In [599]:
mnb.prior

Positif    0.5
Negatif    0.5
Name: label, dtype: float64

In [600]:
mnb.likelihood

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tau,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Negatif,0.001901,0.001901,0.001901,0.001901,0.001901,0.001901,0.003802,0.001901,0.001901,0.005703,0.001901,0.001901,0.001901,0.003802,0.001901,0.003802,0.001901,0.001901,0.001901,0.003802,0.001901,0.001901,0.003802,0.003802,0.003802,0.005703,0.003802,0.003802,0.003802,0.001901,0.003802,0.003802,0.001901,0.003802,0.001901,0.001901,0.003802,0.001901,0.003802,0.001901,...,0.001901,0.001901,0.001901,0.001901,0.001901,0.003802,0.003802,0.003802,0.003802,0.001901,0.001901,0.001901,0.001901,0.001901,0.001901,0.003802,0.003802,0.001901,0.001901,0.001901,0.001901,0.003802,0.001901,0.001901,0.001901,0.001901,0.001901,0.001901,0.001901,0.003802,0.001901,0.003802,0.003802,0.001901,0.003802,0.001901,0.003802,0.001901,0.001901,0.007605
Positif,0.003717,0.001859,0.001859,0.001859,0.003717,0.001859,0.001859,0.001859,0.001859,0.003717,0.001859,0.003717,0.001859,0.001859,0.001859,0.001859,0.003717,0.003717,0.003717,0.001859,0.003717,0.007435,0.001859,0.001859,0.001859,0.001859,0.001859,0.001859,0.001859,0.001859,0.003717,0.001859,0.001859,0.007435,0.001859,0.001859,0.003717,0.003717,0.001859,0.001859,...,0.003717,0.001859,0.001859,0.001859,0.001859,0.001859,0.001859,0.001859,0.003717,0.001859,0.003717,0.001859,0.003717,0.001859,0.003717,0.001859,0.005576,0.001859,0.001859,0.001859,0.001859,0.001859,0.005576,0.001859,0.001859,0.003717,0.003717,0.003717,0.001859,0.001859,0.003717,0.001859,0.001859,0.001859,0.001859,0.003717,0.001859,0.001859,0.001859,0.003717


In [601]:
mnb.posterior

,Negatif,Positif
27,8.078715e-19,6.132168e-18
4,3.231486e-18,6.132168e-19
8,1.654389e-38,2.096849e-39
0,3.814414e-35,5.057669e-35
24,2.034354e-34,7.586503e-35
25,5.863093e-31,5.489670e-30
17,1.838210e-40,8.736870e-41
7,2.417246e-38,3.525327e-38
14,8.976350e-20,6.132168e-19
21,1.203829e-30,1.088410e-30


## Cross Validation

In [602]:
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(tf, df['label'])

5

In [603]:
for train_index, test_index in skf.split(tf, df['label']):
  print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [ 3  4  5  6  7  8  9 10 11 12 13 14 18 19 20 21 22 23 24 25 26 27 28 29] TEST: [ 0  1  2 15 16 17]
TRAIN: [ 0  1  2  6  7  8  9 10 11 12 13 14 15 16 17 21 22 23 24 25 26 27 28 29] TEST: [ 3  4  5 18 19 20]
TRAIN: [ 0  1  2  3  4  5  9 10 11 12 13 14 15 16 17 18 19 20 24 25 26 27 28 29] TEST: [ 6  7  8 21 22 23]
TRAIN: [ 0  1  2  3  4  5  6  7  8 12 13 14 15 16 17 18 19 20 21 22 23 27 28 29] TEST: [ 9 10 11 24 25 26]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 15 16 17 18 19 20 21 22 23 24 25 26] TEST: [12 13 14 27 28 29]
